In [1]:
# imports
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
X_train = pd.read_csv('data/X_train_h1n1.csv', index_col= 'respondent_id')
X_test = pd.read_csv('data/X_test_h1n1.csv', index_col= 'respondent_id')
y_train = pd.read_csv('data/y_train_h1n1.csv', index_col= 'respondent_id')
y_test = pd.read_csv('data/y_test_h1h1.csv', index_col= 'respondent_id')

In [3]:
X_train_cat = X_train.select_dtypes('object')
X_train_num = X_train.select_dtypes(['float64', 'int64'])

cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])
num_pipe = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean'))
])

transformer = ColumnTransformer([('categorical', cat_pipe, X_train_cat.columns),
                                 ('numerical', num_pipe, X_train_num.columns)])

In [4]:
logreg_pipe = imbPipeline([
    ('trans', transformer),
    ('smote', SMOTE(random_state=42)),
    ('logreg', LogisticRegression(n_jobs=-1, random_state=42))
])
logreg_pipe.fit(X_train, y_train)

C:\Users\Manav Kahlon\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa'],
      dtype=...
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'op

# Logistic Regression with Grid Search

In [5]:
logreg_pipe_grid = {'logreg__C': [1e-2, 1, 1e2]}
gs_logreg_pipe = GridSearchCV(estimator=logreg_pipe, param_grid=logreg_pipe_grid,
                              cv=5, verbose=3, n_jobs=-1)
gs_logreg_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


C:\Users\Manav Kahlon\anaconda3\envs\learn-env\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trans',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_...
       'opinion_h1n1_vacc_effective', 'opinion_h1n1

In [6]:
gs_logreg_pipe.best_params_

{'logreg__C': 100.0}

In [7]:
print('f1 Score Train:', gs_logreg_pipe.score(X_train, y_train))
print('f1 Score Test:', gs_logreg_pipe.score(X_test, y_test))

f1 Score Train: 0.7773339990014978
f1 Score Test: 0.775198442414258


In [8]:
log_cv_f1 = cross_val_score(gs_logreg_pipe, X_train, y_train, n_jobs=-1, verbose=3) 
log_cv_f1

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   18.8s remaining:   28.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   19.0s finished


array([0.77034448, 0.77608587, 0.77358962, 0.77109336, 0.78332501])

# KNN

In [9]:
knn_pipe = imbPipeline([
    ('trans', transformer),
    ('smote', SMOTE(random_state=42)),
    ('knn', KNeighborsClassifier(n_jobs=-1))
])
knn_pipe.fit(X_train, y_train)

C:\Users\Manav Kahlon\anaconda3\envs\learn-env\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa'],
      dtype=...
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'op

In [ ]:
knn_pipe_grid = {'knn__n_neighbors': [5, 7, 9], 'knn__p': [1, 2, 3]}
gs_knn_pipe = GridSearchCV(
    estimator=knn_pipe, param_grid=knn_pipe_grid, cv=5, n_jobs=-1, verbose=3)
gs_knn_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [ ]:
gs_knn_pipe.best_params_

In [ ]:
print('f1 Score Train:', gs_knn_pipe.score(X_train, y_train))
print('f1 Score Test:', gs_knn_pipe.score(X_test, y_test))

In [ ]:
knn_cv_score = cross_val_score(gs_knn_pipe, X_train, y_train, n_jobs=-1, verbose=3)
knn_cv_score

KNN was not better than a logistic regression when getting the best f1 score of of the test data.

# Support Vector Classifier

In [ ]:
svm_pipe = imbPipeline([
    ('trans', transformer), 
    ('smote', SMOTE(random_state=42)),
    ('svm', SVC())
])
svm_pipe.fit(X_train, y_train)

In [ ]:
svc_cv_score = cross_val_score(svm_pipe, X_train, y_train, n_jobs = -1, verbose = 3)

In [ ]:
svc_cv_score

# SVC with a Grid Search

In [ ]:
svm_pipe_grid = {'svm__C': [.001, 1, 100, 1000], 'svm__gamma': [0.001, 1, 100]}
gs_svm_pipe = GridSearchCV(estimator=svm_pipe, param_grid=svm_pipe_grid, verbose=1, cv=5, n_jobs=-1, scoring = 'f1')
gs_svm_pipe.fit(X_train, y_train)